### 피어슨 상관계수 - 상관분석

In [1]:
import pandas as pd
from sklearn.datasets import load_diabetes

In [2]:
diabetes=load_diabetes()
data=pd.DataFrame(diabetes.data,columns=diabetes.feature_names)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [4]:
from scipy.stats import pearsonr

In [6]:
pearsonr(x=data['age'],y=data['bmi'])

PearsonRResult(statistic=0.1850846661465555, pvalue=9.076791865417418e-05)

- (상관계수,p-value)  
- 상관계수가 적어서 관계가 적음
- 유의수준 5% 귀무가설 기각  
- 상관계수는 작지만 유의미하다

In [7]:
data[['sex','bmi']].corr()

,sex,bmi
sex,1.000000,0.088161
bmi,0.088161,1.000000


### 샤피로 윌크 검정 - 정규성검정

In [8]:
import numpy as np

np.random.seed(2024)
x=np.random.random(10)

In [9]:
from scipy.stats import shapiro
shapiro(x)

ShapiroResult(statistic=0.8969663381576538, pvalue=0.20285381376743317)

- (통계량,p-value)형태 
- p-value가 커서 정규성을 만족하지 못함

## 모평균과 모분산 검정
### 단일표본 t검정
- ttest_1samp(list,popmean)
- defalut: popmean 과 다름

In [2]:
import numpy as np

kg=np.array([75.5,83.9,75.7,56.2,73.4,67.7,79.0,50.7,58.4,74.1,65.1,77.8,48.1,46.3])

np.mean(kg)

66.56428571428572

In [3]:
# 단일표본 t검정 실행
from scipy.stats import ttest_1samp
ttest_1samp(kg,70)

TtestResult(statistic=-1.0289933120202257, pvalue=0.3222484823978743, df=13)

- (t통계량값,p-value)
- p-value 5프로 이상이라 귀무가설을 기각을 못함
- 모집단 평균은 70이 아니다.

### 대응표본 t검정
- ttest_1samp(list,list)
- defalut : 두 표본의 평균의 차이가 0이 아님

In [5]:
import pandas as pd
female=np.array([50.7,58.4,74.1,65.1,77.8,48.1,46.3])
male=np.array([75.5,83.9,75.7,56.2,73.4,67.7,79.0])
diff=female-male
np.mean(diff)

-12.985714285714291

In [6]:
from scipy.stats import ttest_rel
ttest_rel(female,male)

TtestResult(statistic=-2.078446933064972, pvalue=0.08291274205610201, df=6)

- (t통계량,p-value)
- 0.83>5% 귀무가설 기각x -> 모평균이 다르다

### 독립표본 t-검정
- ttest_ind(list,list)
- 등분산인지 여부
- 두 표본의 평균의 0이 아님